In [2]:
# ==========================================
# Customer Support Ticket Processing System
# ==========================================

import pandas as pd
import re
from datetime import timedelta

# ==========================================
# 1. Load & Explore Dataset
# ==========================================
input_file = "customer_support_tickets.csv"  # keep CSV in same folder
df = pd.read_csv(input_file)

print("Rows, Columns:", df.shape)
print("\nMissing Values:\n", df.isnull().sum())
print("\nSample Tickets:\n", df.head())

# ==========================================
# 2. Clean Ticket Messages
# ==========================================
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_message"] = df["Ticket Description"].apply(clean_text)

# ==========================================
# 3. Issue Classification (Rule-Based)
# ==========================================
def classify_issue(text):
    if any(k in text for k in ["payment", "card", "upi", "charged", "debit"]):
        return "PAYMENT"
    elif any(k in text for k in ["login", "signin", "password", "otp", "account access"]):
        return "LOGIN"
    elif any(k in text for k in ["delivery", "late", "shipment", "tracking"]):
        return "DELIVERY"
    elif any(k in text for k in ["refund", "return", "money back"]):
        return "REFUND"
    elif any(k in text for k in ["bug", "error", "crash", "issue", "problem"]):
        return "BUG"
    else:
        return "GENERAL"

df["issue_type"] = df["clean_message"].apply(classify_issue)

# ==========================================
# 4. Priority Assignment (P0 – P3)
# ==========================================
def assign_priority(text):
    if any(k in text for k in ["urgent", "immediately", "asap", "not working"]):
        return "P0"
    elif any(k in text for k in ["failed", "error", "charged twice"]):
        return "P1"
    elif any(k in text for k in ["delay", "late", "slow"]):
        return "P2"
    else:
        return "P3"

df["priority"] = df["clean_message"].apply(assign_priority)

# ==========================================
# 5. SLA Hours & Due Time Calculation
# ==========================================
sla_map = {
    "P0": 2,    # Critical
    "P1": 6,    # High
    "P2": 24,   # Medium
    "P3": 72    # Low
}

df["sla_hours"] = df["priority"].map(sla_map)

# Using Date of Purchase as ticket creation time
df["created_time"] = pd.to_datetime(df["Date of Purchase"], errors="coerce")

df["due_time"] = df["created_time"] + df["sla_hours"].apply(
    lambda h: timedelta(hours=h)
)

# ==========================================
# 6. Support Manager Report
# ==========================================
manager_report = (
    df.groupby(["issue_type", "priority"])
      .size()
      .reset_index(name="ticket_count")
)

print("\nSupport Manager Report:\n")
print(manager_report)

# ==========================================
# 7. Export Final CSV
# ==========================================
output_file = "final_support_tickets_processed.csv"
df.to_csv(output_file, index=False)

print("\n✅ Process completed successfully!")
print("📁 Output file saved as:", output_file)


Rows, Columns: (8469, 17)

Missing Values:
 Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64

Sample Tickets:
    Ticket ID        Customer Name              Customer Email  Customer Age  \
0          1        Marisa Obrien  carrollallison@example.com            32   
1          2         Jessica Rios    clarkeashley@example.com            42   
2          3  Christopher Robbins   gonzalestracy@example.c